# Regular grids run simulations

This notebook was used to run simulations in order to produce heatmaps with 4-25 habitat patches in regular grid landscapes.
For the given parameter set simulations are run for four landscapes of different sizes (= number of patches), each with and without evolution for all combinations of connectivity and heterogeneity.

In short the algorithm is:
1. Set landscape (habitat positions and delta matrix).
2. Loop through levels of heterogeneity and set plant carrying capacities for the respective level of heterogeneity.
3. Loop through maximum dispersal distances.
4. Run simulation without and with evolution and save pollinator extinction time.


All output and information to reproduce the exact time series (initial values, random seed, landscapes) are available in "Data".

To run new simulations: Copy folder "0" under "Data/grid", rename with new replication count. Then set the same replication count at top of code cell 4 and run the whole notebook. Important: Make sure not to select a replication count already taken to not overwrite data!

In [1]:
# load needed packages
import random as rd
import numpy as np
import pandas as pd
from scipy import integrate as integ
import spatial_eco_evo_functions_grid as fct_grid

# Parameter set

In [2]:
# set ecological parameter values:
gamma_A = 0.2            # interaction benefit provided by the pollinator
gamma_P = 1              # interaction benefit provided by the pollinator
c_A = 1                  # intraspecific competition strength of the pollinator 
s = 2.5                  # trade-off shape form factor

# dispersal rate
d = 0.2

# initial pollinator growth rate
r_Aorig = 0.6            # for comparison with non-spatial model
r_A0 = r_Aorig + d

# parameter for environmental change
r_A_change = -0.000008   # change of r_A per time step

# parameters for evolution
t_interval = 20           # length of time interval between mutations
t_step = 1                # one data point per time step
mut_step = 0.02           # maximum mutation step size

# extinction threshold
A_ext = 0.0001

# Heatmaps

In [3]:
# overall settings and arrays/lists to loop through
environmental_change = True
t_end = 400000           # total runtime

N_vals = [4, 9, 16, 25]
h_vals = np.arange(0, 2.1, 0.2) # degree of heterogeneity
delta_max_vals = np.arange(0, 1.1, 0.1) # maximum dispersal distance

In [4]:
#rep = 20 # replica count


for N in N_vals:
    '''########### set LANDSCAPE ############'''
    x_coord, y_coord, delta, delta_boundary, neighbours = fct_grid.landscape_grid(N)
    
    ####################################################################################
    ########### parameters ########### (do not need to save coordinates and delta as it is allways the same)
    params_save = pd.DataFrame({'N': [N], 'd': [d], 's': [s],
                            'gamma_A': [gamma_A], 'gamma_P': [gamma_P], 
                            'c_A': [c_A], 'r_A_change': [r_A_change], 
                            't_interval': [t_interval], 'mut_step': [mut_step]})
    
    params_save.to_excel("Data/grid/" + str(rep) + "/N=" + str(N) + "/N=" 
                    + str(N) + ",lsc=grid.xlsx", sheet_name='Parameters')
    ####################################################################################
    
    
    # open empty matrices to store the extinction times (two heatmaps parallel)
    extinction_times_tot_evo = np.zeros((len(delta_max_vals),len(h_vals)))
    extinction_times_tot = np.zeros((len(delta_max_vals),len(h_vals)))

    
    '''########### Loop through HETEROGENEITY AND CONNECTIVITY values ###########'''
    for k, h in enumerate(h_vals): # loop through heterogeneity values -> y-axis = rows

        # draw carrying capacities and set competition strength
        c_P_vals = fct_grid.c_P_values(N, h)
        
        ####################################################################################
        ############  save competition coefficients ###########
        c_P_save = pd.DataFrame(data = c_P_vals)
        c_P_save.to_csv("Data/grid/" + str(rep) + "/N=" + str(N) + "/c_P/c_P, N=" 
                        + str(N) + ",h=" + str(h) + ",lsc=grid.csv", index=False)
        ####################################################################################
        
        
        for l, delta_max in enumerate(delta_max_vals): # loop through connectivity values -> x-axis = columns

            # calculate linkwise success
            success = fct_grid.linkwise_success(delta_boundary, delta_max, neighbours)
            
            # calculate migration frations
            fractions = fct_grid.fractions_ij(success, neighbours)
            
            # calculation of migration matrix
            mig_matrix = fct_grid.migration_matrix(success, fractions)
            
            # population initialisation
            x0_ini, alpha_max_vals, alpha_r_vals_ini, alpha_m_vals_ini = fct_grid.initialisation(c_P_vals, c_A, gamma_P, gamma_A, 
                                                                                s, d, r_A0, r_A_change, N, mig_matrix, 0.02)
            
            # set draw random seed for evolution
            seeding = rd.randint(0, 1000000000)
            
            ####################################################################################
            ############### Save initial population densities, alpha_values and seed ###########
            x0_save = pd.DataFrame({'x0': x0_ini})
            alpha_save = pd.DataFrame({'alpha_max_vals': alpha_max_vals, 
                                       'alpha_r_vals': alpha_r_vals_ini,
                                       'alpha_m_vals': alpha_m_vals_ini})
            seeding_save = pd.DataFrame({'Seed': [seeding]})

            with pd.ExcelWriter("Data/grid/" + str(rep) + "/N=" + str(N) + "/initial/init, N=" 
                                + str(N) + ",delta_max=" + str(delta_max) + ",h=" + str(h) + ",lsc=grid.xlsx") as writer:  
                x0_save.to_excel(writer, sheet_name='Densities')
                alpha_save.to_excel(writer, sheet_name='Alpha values')
                seeding_save.to_excel(writer, sheet_name='Seed')
            ####################################################################################
            
            '''SIMULATIONS for heatmap with and without evolution'''
            for evolution in [True, False]:
                # prepare empty lists to store simulation results for extinction checks
                x_complete = []
                
                # set initial values & seed
                x0 = x0_ini
                alpha_r_vals = alpha_r_vals_ini
                alpha_m_vals = alpha_m_vals_ini
                rd.seed(seeding)

                
                for i in range (int(t_end / t_interval)):       # loop through the time intervals

                    # calculate per-capita growth rates from alpha
                    r_P_r_vals = fct_grid.r_P_function(alpha_r_vals, alpha_max_vals, s)
                    r_P_m_vals = fct_grid.r_P_function(alpha_m_vals, alpha_max_vals, s)

                    # t-values for the current time interval, that is between pertubation i and pertubation i+1:
                    t = np.arange(0,t_interval,t_step) + i*t_interval

                    # additional parameters to give into the odeint-function
                    env_params = [r_A0, r_A_change, environmental_change]
                    plant_params = [alpha_r_vals, r_P_r_vals]
                    migration_params =  [d, mig_matrix]

                    # time series
                    x = integ.odeint(fct_grid.metacommunity,x0,t, 
                                     args=(env_params, c_A, c_P_vals, gamma_A, gamma_P, plant_params, migration_params, N))
                    
                    
                    # check for pollinator extinction:
                    x[-1] = np.where(x[-1] < A_ext, 0, x[-1]) # when below extinction threshold send to 0

                    extinct = np.array(x[-1,::2] < A_ext) # one value per pollinator population: False = alive, True = extinct
                    if np.all(extinct): # when all local populations are extinct save the time
                        t_extinct = t[-1]
                        
                        if evolution:
                            extinction_times_tot_evo[k,l] = t_extinct
                        else:
                            extinction_times_tot[k,l] = t_extinct
                    
                    
                    # environmental change for evolution
                    if environmental_change:
                        r_A = r_A0 + t * r_A_change  
                    else:
                        r_A = r_A0 + t * 0
                    
                    
                    # perform evolution if activated
                    if evolution:
                        alpha_r_vals, alpha_m_vals = fct_grid.evolution_function(alpha_r_vals, alpha_m_vals, 
                                                                            r_P_r_vals, r_P_m_vals, 
                                                                            c_P_vals, r_A[-1], 
                                                                            c_A, gamma_P, gamma_A, s, 
                                                                            mut_step, d, x[-1])
                    
                    # save densities
                    x0 = x[-1]    
                    x_complete.extend(x)
                    

                    if np.all(extinct): # if I want to stop simulation when pollinators are extinct
                        break
                
    ####################################################################################
    ######### save heatmap data extinction times ###########
    for evolution in [True, False]:
        if evolution:
            ext_times_save = pd.DataFrame(data = extinction_times_tot_evo)
        else:
            ext_times_save = pd.DataFrame(data = extinction_times_tot)
        
        ext_times_save.to_csv("Data/grid/" + str(rep) + "/N=" + str(N) + "/t_ext, N=" 
                      + str(N) + ",evo=" + str(evolution) + ",lsc=grid.csv", index=False)
    ####################################################################################